# Домашнее задание по теме «Ансамблирование»


    Теперь решаем задачу регрессии - предскажем цены на недвижимость. Использовать датасет https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data (train.csv)
    1) Данных немного, поэтому необходимо использовать 10-fold кросс-валидацию для оценки качества моделей
    2) Построить случайный лес, вывести важность признаков
    3) Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную
    4) Для валидации модели 2-го уровня использовать отдельный hold-out датасет, как на занятии
    5) Показать, что использование ансамблей моделей действительно улучшает качество (стекинг vs другие модели сравнивать на hold-out)
    6) В качестве решения: Jupyter notebook с кодом, комментариями и графиками, ссылка на гитхаб

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import auc, roc_curve, roc_auc_score
%matplotlib inline

In [2]:
data = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
X = data.drop(['Id', 'SalePrice'], axis=1)
Y = data.SalePrice
X.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
#Находим категориальные признаки
cat_feat = list(X.dtypes[data.dtypes == object].index)

#закодируем пропущенные значений строкой, факт пропущенного значения тоже может нести в себе информацию
X[cat_feat] = X[cat_feat].fillna('nan')
d_train, d_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state= 666)

#отфильтруем непрерывные признаки
num_feat = [f for f in X if f not in (cat_feat + ['ID', 'target'])]

# Смотрим сколько у нас значений по каждому категориальному признаку
cat_nunique = X[cat_feat].nunique()
#print(cat_nunique)

#Чтобы в разы не увеличивать число признаков при построении dummy,
#будем использовать категориальные признаки с < 30 уникальных значений
cat_feat = list(cat_nunique[cat_nunique < 30].index)
#cat_feat

In [6]:
# Создаем дамми-переменные для категорий

dummy_train = pd.get_dummies(d_train[cat_feat], columns=cat_feat)
dummy_test = pd.get_dummies(d_test[cat_feat], columns=cat_feat)

dummy_cols = list(set(dummy_train) & set(dummy_test))

dummy_train = dummy_train[dummy_cols]
dummy_test = dummy_test[dummy_cols]


# Заменяем пропуски на специальное значение -999, чтобы деревья могли их отличить
X_train = pd.concat([d_train[num_feat].fillna(-999),
                     dummy_train], axis=1)

X_test = pd.concat([d_test[num_feat].fillna(-999),
                     dummy_test], axis=1)

In [7]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor

In [8]:
#массив с оченками всех моделей
score=[]

### Оценка обычного дерева

In [9]:
clf_tree = DecisionTreeRegressor(max_depth=15, min_samples_leaf=20)
clf_tree.fit(X_train, y_train)
#оценка обычного дерева
score.append(['DecisionTreeRegressor',clf_tree.score(X_test, y_test)])
score[-1:]

[['DecisionTreeRegressor', 0.7376346876969766]]

### Оценка Беггинг модели

In [10]:
bag_clf = BaggingRegressor(n_estimators=10, base_estimator=clf_tree, n_jobs=-1)
bag_clf.fit(X_train, y_train)

score.append(['BaggingRegressor',bag_clf.score(X_test,y_test)])
score[-1:]

[['BaggingRegressor', 0.7493833202732039]]

# Random Forest

### Оценка Random Forest модели

In [11]:
clf_rf = RandomForestRegressor(n_estimators=10, n_jobs=-1)
clf_rf.fit(X_train, y_train)
score.append(['RandomForestRegressor',clf_rf.score(X_test,y_test)])
score[-1:]

[['RandomForestRegressor', 0.8459131691228122]]

### 10-fold кросс-валидация

In [12]:
from sklearn.model_selection import StratifiedKFold

y_pred_all = np.zeros_like(y_test, dtype=float)
    
stack_cv = StratifiedKFold(n_splits=10, random_state=666)
    
for i, (train_ind, test_ind) in enumerate(stack_cv.split(X_train, y_train)):
    clf_rf.fit(X_train.iloc[train_ind], y_train.iloc[train_ind])
    y_pred_all += clf_rf.predict(X_test)

y_pred = y_pred_all / stack_cv.n_splits

C:\Users\sidorov\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


In [13]:
from sklearn.metrics import r2_score

score.append(['10-folk CV RandomForest',r2_score(y_test, y_pred)])
score[-1:]

[['10-folk CV RandomForest', 0.8397620882724911]]

### Оценка значимости фич

In [14]:
# сделаем на всей выборке
X1 = pd.concat([X_train, X_test])
y1 = pd.concat([y_train, y_test])

In [15]:
clf_rf = RandomForestRegressor(n_estimators=10, n_jobs=-1)
clf_rf.fit(X1, y1)   

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [16]:
feature_importances=pd.DataFrame(zip(X_train.columns,clf_rf.feature_importances_), columns=['Feature', 'feature_importance'])
feature_importances.sort_values('feature_importance', ascending=False).head(10)

,Feature,feature_importance
3,OverallQual,0.609039
15,GrLivArea,0.107741
8,BsmtFinSF1,0.035730
13,2ndFlrSF,0.026285
11,TotalBsmtSF,0.025683
12,1stFlrSF,0.017516
18,FullBath,0.015851
5,YearBuilt,0.015549
2,LotArea,0.012241
1,LotFrontage,0.011441


Больше всего на стоимость дома влияет общее качество материалов и отделки.

# Стекинг из 3-х моделей

In [17]:
def get_meta_features(clf, X_train, y_train, X_test, stack_cv):
    meta_train = np.zeros_like(y_train, dtype=float)
    meta_test = np.zeros_like(y_test, dtype=float)
    
    for i, (train_ind, test_ind) in enumerate(stack_cv.split(X_train, y_train)):
        
        clf.fit(X_train.iloc[train_ind], y_train.iloc[train_ind])
        meta_train[test_ind] = clf.predict(X_train.iloc[test_ind])
        meta_test += clf.predict(X_test)
    
    return meta_train, meta_test / stack_cv.n_splits

In [18]:
from sklearn.preprocessing import StandardScaler

train_median = d_train[num_feat].median()

X_train_lin = pd.concat([d_train[num_feat].fillna(train_median),
                     d_train[num_feat + cat_feat].isnull().astype(np.int8).add_suffix('_NaN'),
                     dummy_train], axis=1)

X_test_lin = pd.concat([d_test[num_feat].fillna(train_median),
                     d_test[num_feat + cat_feat].isnull().astype(np.int8).add_suffix('_NaN'),
                     dummy_test], axis=1)

scaler = StandardScaler()
scaler.fit(X_train_lin[num_feat])

X_train_lin[num_feat] = scaler.transform(X_train_lin[num_feat])
X_test_lin[num_feat] = scaler.transform(X_test_lin[num_feat])

C:\Users\sidorov\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\sidorov\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  app.launch_new_instance()
C:\Users\sidorov\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.


In [19]:
from sklearn.linear_model import LinearRegression
clf_lr = LinearRegression()
clf_lr.fit(X_train_lin[num_feat],y_train)
score.append(['LinearRegression',clf_lr.score(X_test_lin[num_feat], y_test)])
score[-1:]

[['LinearRegression', 0.7786453332987588]]

In [20]:
from sklearn.model_selection import StratifiedKFold

stack_cv = StratifiedKFold(n_splits=10, random_state=666)

meta_train = []
meta_test = []
col_names = []

print('LR features...')
meta_tr, meta_te = get_meta_features(clf_lr, X_train_lin, y_train, X_test_lin, stack_cv)

meta_train.append(meta_tr)
meta_test.append(meta_te)
col_names.append('lr_pred')

print('RF features...')
meta_tr, meta_te = get_meta_features(clf_rf, X_train, y_train, X_test, stack_cv)

meta_train.append(meta_tr)
meta_test.append(meta_te)
col_names.append('rf_pred')

print('Bagging features...')
meta_tr, meta_te = get_meta_features(bag_clf, X_train, y_train, X_test, stack_cv)

meta_train.append(meta_tr)
meta_test.append(meta_te)
col_names.append('bag_pred')

C:\Users\sidorov\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


LR features...
RF features...


C:\Users\sidorov\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Bagging features...


C:\Users\sidorov\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


In [21]:
X_meta_train = pd.DataFrame(np.stack(meta_train, axis=1), columns=col_names)
X_meta_test = pd.DataFrame(np.stack(meta_test, axis=1), columns=col_names)

In [22]:
X_meta_test.head()

,lr_pred,rf_pred,bag_pred
0,146531.25,144532.97,142515.770938
1,199952.10,189039.56,193176.438192
2,215582.05,255362.70,257277.129205
3,144425.80,145012.65,142543.894852
4,102569.80,109088.00,108264.907387


In [23]:
clf_lr_meta = LinearRegression()
clf_lr_meta.fit(X_meta_train, y_train)

score.append(['x3 model stack',clf_lr_meta.score(X_meta_test, y_test)])
score[-1:]

[['x3 model stack', 0.8420848734735801]]

# Оценки моделей

In [24]:
pd.DataFrame(score, columns = ['Model', 'Score'])

,Model,Score
0,DecisionTreeRegressor,0.737635
1,BaggingRegressor,0.749383
2,RandomForestRegressor,0.845913
3,10-folk CV RandomForest,0.839762
4,LinearRegression,0.778645
5,x3 model stack,0.842085


In [25]:
clf_lr_meta.coef_

array([-6.45417070e-13,  1.34408547e+00, -1.79229032e-01])